In [1]:
import gzip
import random
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

In [3]:
example = readCSV("train_Interactions.csv.gz")
print(next(example))
del example

['u79354815', 'b14275065', '4']


# Question 9

In [4]:
### Ratings Prediction

train_size = 150000
val_size   = 10000
data       = [line for line in readCSV("train_Interactions.csv.gz")]
train      = data[:train_size]
val        = data[train_size:]

allRatings = []
userBookRatings = defaultdict(lambda: defaultdict(float))
userRatings = defaultdict(list)
userBooks   = defaultdict(set)
bookUsers   = defaultdict(set)

for user, book, rating in train:
    rating = int(rating)
    allRatings.append(rating)
    userRatings[user].append(rating)
    userBookRatings[user][book] = rating
    userBooks[user].add(book)
    bookUsers[book].add(user)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for user in userRatings:
    userAverage[user] = sum(userRatings[user]) / len(userRatings[user])

In [5]:
# Coordinate Descent

def coordinate_descent(lambda_opt = 1, iterations = 100):

    alpha_sum, bu_sum, bb_sum = 0, 0, 0

    train_len = len(train)
    bu = defaultdict(lambda: 1)
    bb = defaultdict(lambda: 1)

    for descent in range(iterations):
        alpha_sum = 0
        for user, book, _ in train:
            alpha_sum += userBookRatings[user][book] - (bu[user] + bb[book])
        alpha = alpha_sum / train_len

        for user in userRatings:
            bu_sum = 0
            for book in userBooks[user]:
                bu_sum += userBookRatings[user][book] - (alpha + bb[book])
            bu[user] = bu_sum / (lambda_opt + len(userBooks[user]))

        for book in bookUsers:
            bb_sum = 0
            for user in bookUsers[book]:
                bb_sum += userBookRatings[user][book] - (alpha + bu[user])
            bb[book] = bb_sum / (lambda_opt + len(bookUsers[book]))
            
        if descent % 10 == 0:
            rating_labels = []    
            diff = 0
            for user, book, rating in val:
                if user in all_users_lst and book in all_books_lst: 
                    user_ind = all_users_lst.index(user)
                    book_ind = all_books_lst.index(book)
                    user_rating = alpha + bu[user] + bb[book] + gamma_u[user_ind] * gamma_b[book_ind]
                else:
                    user_rating = alpha + bu[user] + bb[book];
                diff += (user_rating - int(rating)) ** 2

            MSE = diff / len(val)
            print('MSE on the validation set', MSE)
            
    return alpha, bu, bb

alpha, bu, bb = coordinate_descent(lambda_opt = 1)

NameError: name 'all_users_lst' is not defined

In [ ]:
rating_labels = []
diff = 0
for user, book, rating in val:
    user_rating = alpha + bu[user] + bb[book]
    diff += (user_rating - int(rating)) ** 2

MSE = diff / len(val)
print(MSE)

# Question 10

# TODO

# Question 11

In [ ]:
lambda_values = np.logspace(-1, 1.5, num = 20)
print(lambda_values)

In [ ]:
MSEs = []
loop_count = 0
for lambda_opt in lambda_values:
    loop_count += 1; print(loop_count, end = ', ')
    alpha, bu, bb = coordinate_descent(lambda_opt, iterations = 20)
    rating_labels = []
    diff = 0
    for user, book, rating in val:
        user_rating = alpha + bu[user] + bb[book]
        diff += (user_rating - int(rating)) ** 2
    MSE = diff / len(val)
    MSEs.append(MSE)

In [ ]:
plt.plot(lambda_values, MSEs, label='Validation')
plt.ylabel('MSE')
plt.xlabel('lambda value'), plt.xscale('log')
plt.title('MSE vs lambda values')
plt.legend()
plt.show()

In [ ]:
indx = MSEs.index(min(MSEs))
print('\nLambda for lowest MSE is:', lambda_values[indx])
print('\nBest MSE is:', MSEs[indx])

In [ ]:
alpha, bu, bb = coordinate_descent(2.8, iterations = 100)

In [ ]:
with open("predictions_Rating.txt", 'w') as predictions:
    for l in open("pairs_Rating.txt"):
        if l.startswith("userID"):
            #header
            predictions.write(l)
            continue
        user, book = l.strip().split('-')
        user_rating = alpha + bu[user] + bb[book]
        predictions.write(user + '-' + book + ',' + str(user_rating) + '\n')